In [1]:
from scipy.optimize import linprog

# Змінні:
# x[0] — темп стрільби зброї A (залпів за хвилину)
# x[1] — темп стрільби зброї B (залпів за хвилину)
# x[2] — темп стрільби зброї C (залпів за хвилину)

# Цільова функція — максимізувати загальну кількість залпів:
# Зброя A: 3 хв * x[0], B: 5 хв * x[1], C: 4 хв * x[2] — треба мінімізувати від'ємне значення
c = [-3, -5, -4]

# Матриця обмежень (ліва частина A_ub * x <= b_ub)
A = [
    [3, 2, 4],    # Загальна кількість залпів у межах 15
    [2, 3, 0],    # Обмеження на інший період ≤ 8
    [1, 0, -1]    # Різниця A (3 хв) та C (1 хв) ≤ 5
]

b = [
    15,  # загальна кількість залпів
    8,   # додаткове обмеження
    5    # різниця у вогневій потужності
]

# Межі змінних (усі значення мають бути ≥ 0)
x_bounds = [(0, None), (0, None), (0, None)]

# Виконання оптимізації
res = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Виведення результатів
if res.success:
    print("Оптимальний темп стрільби:")
    print(f"Зброя A: {res.x[0]:.2f} залпів/хв")
    print(f"Зброя B: {res.x[1]:.2f} залпів/хв")
    print(f"Зброя C: {res.x[2]:.2f} залпів/хв")
    print(f"Максимальна загальна кількість залпів: {-res.fun:.2f}")
else:
    print("Рішення не знайдено:", res.message)


Оптимальний темп стрільби:
Зброя A: 0.00 залпів/хв
Зброя B: 2.67 залпів/хв
Зброя C: 2.42 залпів/хв
Максимальна загальна кількість залпів: 23.00
